In [1]:
import urllib.request
import json
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from kaggle.api.kaggle_api_extended import KaggleApi

## IMDb

Next we further complement our data with the raitings from each episode of The Office, for all the 9 seasons using IMDb:https://www.imdb.com/title/tt0386676/episodes/_ajax.


IMDb doesn't have official API, so have found the id of The Office in IMDb: `tt0386676`, next we query the data for each episode:

In [2]:
imdb_url =  "https://www.imdb.com/title/tt0386676/episodes/_ajax"
number_of_seasons = 9

We use the `requests` library instead of `urllib3`, and the python `BeautifulSoup` library, as we are making ajax requests.

We make one request for each of the 9 seasons. For each request we extract: the season raitings, titles, votes, air dates and descriptions.

The result is a data set consisting of 7 attributes, namely: the season of the show, episode number for each rating, title for each episode, IMDb raiting, number of votes, air date and description of the episode.

In [3]:
def get_raitings_and_votes(soup):
    ratings = []
    votes = []
    episodes = []
    
    rating_divs = soup.findAll("div", {"class": "ipl-rating-widget"})
    
    for index, div in enumerate(rating_divs):
        episodes.append(index + 1)

        # Find IMDb rating
        rating_div_inner = div.findAll("div", {"ipl-rating-star small"})
        soup_inner_rating = rating_div_inner[0].findAll("span", {"ipl-rating-star__rating"})
        ratings.append(soup_inner_rating[0].string)

        # Find total votes
        soup_inner_votes = rating_div_inner[0].findAll("span", {"ipl-rating-star__total-votes"})
        votes_string = soup_inner_votes[0].string
        votes_string = votes_string.replace(',', '')
        votes_string = votes_string.replace('(', '')
        votes_string = votes_string.replace(')', '')
        votes.append(int(votes_string))
        
    return episodes, ratings, votes

In [4]:
def get_episode_titles(soup):
    titles = []
    title_divs = soup.findAll("strong")
        
    for div in title_divs:
        titles.append(div.string) 
        
    #Popping the extra title (eg Season 1, Season 2, etc) at end for each season (not required)
    titles.pop()
    
    return titles

In [5]:
def get_airdates(soup):
    airdates = []
    
    airdate_divs = soup.findAll("div", {"class": "airdate"})
    for div in airdate_divs:
        airdate_string = div.string
        airdate_string = airdate_string.replace('.', '')
        airdate_string = airdate_string.strip()
        airdates.append(airdate_string)
        
    return airdates

In [6]:
def get_episodes_description(soup):
    desciptions = []
    
    description_divs = soup.findAll("div", {"class": "item_description"})

    for description in description_divs:
        description_string = description.string.strip()
        desciptions.append(description_string)
    
    return desciptions

In [7]:
imdb_df = pd.DataFrame([])

for season in range(1, number_of_seasons + 1):
    print(f'Finding IMDb data for season {season}...')

    r = requests.get(url = imdb_url, params = {'season': season})
    
    # URL response is in HTML format
    soup = BeautifulSoup(r.content, 'html.parser')
    
    episodes, ratings, votes = get_raitings_and_votes(soup)
    titles = get_episode_titles(soup)
    airdates = get_airdates(soup)
    descriptions = get_episodes_description(soup)

    number_of_ep = len(ratings)
    seasons = [season] * number_of_ep
    
    # Preparing data for current season    
    data = {'Season': seasons, 'Episode': episodes, 'EpisodeTitle': titles, 'IMDBRating': ratings, 'TotalVotes': votes, 'AirDate': airdates, 'Description': descriptions}
    imdb_season_df = pd.DataFrame(data)
    imdb_df = imdb_df.append(imdb_season_df)

Finding IMDb data for season 1...
Finding IMDb data for season 2...
Finding IMDb data for season 3...
Finding IMDb data for season 4...
Finding IMDb data for season 5...
Finding IMDb data for season 6...
Finding IMDb data for season 7...
Finding IMDb data for season 8...
Finding IMDb data for season 9...


Next we examine the IMDb dataframe:

In [8]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188 entries, 0 to 22
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Season        188 non-null    int64 
 1   Episode       188 non-null    int64 
 2   EpisodeTitle  188 non-null    object
 3   IMDBRating    188 non-null    object
 4   TotalVotes    188 non-null    int64 
 5   AirDate       188 non-null    object
 6   Description   188 non-null    object
dtypes: int64(3), object(4)
memory usage: 11.8+ KB


In [9]:
imdb_df.head()

,Season,Episode,EpisodeTitle,IMDBRating,TotalVotes,AirDate,Description
0,1,1,Pilot,7.4,7134,24 Mar 2005,The premiere episode introduces the boss and s...
1,1,2,Diversity Day,8.3,7033,16 Mar 2005,Michael's off color remark puts a sensitivity ...
2,1,3,Health Care,7.7,5863,5 Apr 2005,Michael leaves Dwight in charge of picking the...
3,1,4,The Alliance,8.0,5673,12 Apr 2005,"Just for a laugh, Jim agrees to an alliance wi..."
4,1,5,Basketball,8.4,6299,19 Apr 2005,Michael and his staff challenge the warehouse ...


## Kaggle dataset: The Office Dataset

We further extend this data with the guest starts, directors and writers that we get from Kaggle and we load it into the `imdb_df` dataframe by extracting the following columns: GuestStarts, Director, Writers. 

In [10]:
api = KaggleApi()
api.authenticate()

api.dataset_download_file('nehaprabhavalkar/the-office-dataset','the_office_series.csv', path='./data/kaggle_data')

False

In [11]:
office_df = pd.read_csv('data/kaggle_data/the_office_series.csv', usecols=['Season', 'EpisodeTitle', 'GuestStars', 'Director', 'Writers', 'Viewership'])
office_df['Writers'] = office_df['Writers'].str.split('|') 
office_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Season        188 non-null    int64  
 1   EpisodeTitle  188 non-null    object 
 2   Viewership    188 non-null    float64
 3   GuestStars    29 non-null     object 
 4   Director      188 non-null    object 
 5   Writers       188 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 8.9+ KB


In [12]:
office_df.head()

,Season,EpisodeTitle,Viewership,GuestStars,Director,Writers
0,1,Pilot,11.2,NaN,Ken Kwapis,"[Ricky Gervais , Stephen Merchant and Greg Dan..."
1,1,Diversity Day,6.0,NaN,Ken Kwapis,[B. J. Novak]
2,1,Health Care,5.8,NaN,Ken Whittingham,[Paul Lieberstein]
3,1,The Alliance,5.4,NaN,Bryan Gordon,[Michael Schur]
4,1,Basketball,5.0,NaN,Greg Daniels,[Greg Daniels]


Now we merge this data set with the IMDb data about raitings using the `Season` and `EpisodeTitle` columns that both the `imdb_df` and `office_df` dataframes have in common. Since some columns are the same, i.e Ratings and Viewership, we will maintain the columns from IMDb.

In [13]:
imdb_df = imdb_df.merge(office_df, on=['Season', 'EpisodeTitle'], how='left')
imdb_df["IMDBRating"] = pd.to_numeric(imdb_df["IMDBRating"], downcast="float")
imdb_df['EpisodeTitle'] = imdb_df['EpisodeTitle'].str.replace("'", '') # remove special characters "'"

Since we want to analyse the impact of Guest stars in the season ratings, we will count the number of guest stars in every episode:

In [14]:
def count_guest_starts(guest_starts):
    if guest_starts == "nan":
        return 0 # some episodes do not have guest starts
    else:
        return len(guest_starts.split(',')) # the guest stars are separated by ,

imdb_df['Number of Guest Stars'] = imdb_df['GuestStars'].apply(lambda x: count_guest_starts(str(x)))

In [15]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188 entries, 0 to 187
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Season                 188 non-null    int64  
 1   Episode                188 non-null    int64  
 2   EpisodeTitle           188 non-null    object 
 3   IMDBRating             188 non-null    float32
 4   TotalVotes             188 non-null    int64  
 5   AirDate                188 non-null    object 
 6   Description            188 non-null    object 
 7   Viewership             188 non-null    float64
 8   GuestStars             29 non-null     object 
 9   Director               188 non-null    object 
 10  Writers                188 non-null    object 
 11  Number of Guest Stars  188 non-null    int64  
dtypes: float32(1), float64(1), int64(4), object(6)
memory usage: 18.4+ KB


In [16]:
imdb_df.head()

,Season,Episode,EpisodeTitle,IMDBRating,TotalVotes,AirDate,Description,Viewership,GuestStars,Director,Writers,Number of Guest Stars
0,1,1,Pilot,7.4,7134,24 Mar 2005,The premiere episode introduces the boss and s...,11.2,NaN,Ken Kwapis,"[Ricky Gervais , Stephen Merchant and Greg Dan...",0
1,1,2,Diversity Day,8.3,7033,16 Mar 2005,Michael's off color remark puts a sensitivity ...,6.0,NaN,Ken Kwapis,[B. J. Novak],0
2,1,3,Health Care,7.7,5863,5 Apr 2005,Michael leaves Dwight in charge of picking the...,5.8,NaN,Ken Whittingham,[Paul Lieberstein],0
3,1,4,The Alliance,8.0,5673,12 Apr 2005,"Just for a laugh, Jim agrees to an alliance wi...",5.4,NaN,Bryan Gordon,[Michael Schur],0
4,1,5,Basketball,8.4,6299,19 Apr 2005,Michael and his staff challenge the warehouse ...,5.0,NaN,Greg Daniels,[Greg Daniels],0


#### Save data to csv (to be used in the main notebook)

In [19]:
imdb_df.to_csv("data/imdb_ratings.csv",index=False)